In [61]:
#투표 기반 분류기 (page. 246)
#말 그대로 여러 분류기의 예측을 투표로 받아 가장 많이 선택된 클래스로 예측함
#사이킷런의 투표 기반 분류기
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

#moons 데이터셋 가져오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X, y = make_moons(n_samples=1000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
                             voting='hard')
voting_clf = voting_clf.fit(X_train, y_train)

#각 분류기의 테스트셋 정확도 확인
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.788
RandomForestClassifier 0.808
SVC 0.804
VotingClassifier 0.816


In [62]:
#voting=soft - 분류기가 클래스의 확률 예측할 수 있으면(predict_proba)
#개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True) #확률 예측 가능 (predict_proba 메서드 사용 가능)

voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
                             voting='soft')
voting_clf = voting_clf.fit(X_train, y_train)

#각 분류기의 테스트셋 정확도 확인
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.788
RandomForestClassifier 0.816
SVC 0.804
VotingClassifier 0.82


In [64]:
#배깅(중복 허용), 페이스팅(중복 x) - 무작위로 샘플링하여 여러 개의 예측기를 훈련
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(),
                            n_estimators=500, max_samples=100,
                            bootstrap=True, n_jobs=-1) #bootstrap=False 하면 배깅 대신 페이스팅
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.824


In [65]:
#단 한 번도 선택되지 않은 37%의 샘플들 - out of bag, oob
bag_clf = BaggingClassifier(DecisionTreeClassifier(),
                            n_estimators=500, bootstrap=True,
                            n_jobs=-1, oob_score=True) #oob_score
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.8386666666666667

In [67]:
bag_clf.oob_decision_function_
#각 훈련 샘플의 클래스 확률

array([[0.74011299, 0.25988701],
       [0.735     , 0.265     ],
       [0.15425532, 0.84574468],
       ...,
       [0.25294118, 0.74705882],
       [0.03333333, 0.96666667],
       [0.98979592, 0.01020408]])

랜덤 패치(훈련 특성과 샘플 모두 샘플링),
그리고 랜덤 서브스페이스 - 훈련 샘플을 모두 사용하고(bootstrap=False, max_samples=1.0)
특성은 샘플링(bootstrap_features=True 그리고/또는 max_features= 1.0보다 작게 설정)
BaggingClassifier는 특성 샘플링도 지원
매개변수는 max_features, bootstrap_features

특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춤

In [70]:
#랜덤 포레스트 - 배깅/페이스팅을 적용한 결정 트리의 앙상블
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
#n_estimators는 숲을 이루는 tree의 수
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_rf))

0.808


In [72]:
#위를 BaggingClassifier로 유사하게 만든 것
bag_clf = BaggingClassifier(DecisionTreeClassifier(max_features='auto', max_leaf_nodes=16),
                           n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)

y_pred_bag = rnd_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_bag))

0.808


In [73]:
#익스트림 랜덤 트리 앙상블 (엑스트라 트리) - 극단적으로 무작위한 트리의 랜덤 포레스트
#랜덤 포레스트의 장점 - 특성 중요도 측정이 쉬움 feature_importances_
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])

for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09495566434345105
sepal width (cm) 0.02610340581159169
petal length (cm) 0.4401680297313323
petal width (cm) 0.4387729001136249


In [ ]:
#부스팅 - 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법
#이전 예측기를 보완하는 새로운 예측기를 만드는 방법 - 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것
#에이다부스트:
#1. 알고리즘의 기반이 되는 첫 분류기를 훈련 세트에서 훈련시키고 예측을 만듦
#2. 알고리즘이 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높임
#3. 두 번째 분류기는 업데이트된 가중치를 사용해 훈련 세트에서 훈련하고 다시 예측을 만듦
#4. 그다음 다시 가중치를 업데이트 - 계속 반복

